# Reporting Dashboard Setup for Tableau

This notebook prepares the final engineered dataset for Tableau visualization by loading it into PostgreSQL. Tableau will then connect to this data source to build interactive fraud detection dashboards, enabling stakeholders to explore fraud trends, risk scores, and claim characteristics with ease.

In [1]:
# Imports essential libraries and Setup
import pandas as pd
import os
from sqlalchemy import create_engine, text
from getpass import getpass  # For secure password input

# Project directory and engineered features file path
project_dir = r"C:\Users\Cloud\OneDrive\Desktop\Fraud_Analytics_Project"
feature_file = os.path.join(project_dir, "data", "features", "engineered_insurance_claims.csv")
print("Imports and file path setup complete.")

Imports and file path setup complete.


### 1. PostgreSQL Connection Setup

I created a secure connection to the PostgreSQL database where Tableau will fetch the data.  
Using `getpass()` avoids hardcoding sensitive passwords in code.

In [2]:
# Securely prompt for PostgreSQL password
pg_user = "postgres"
pg_password = getpass("Enter PostgreSQL password: ")
pg_host = "localhost"
pg_port = 5432
pg_dbname = "fraud_analytics"

# Create SQLAlchemy engine for PostgreSQL connection
engine = create_engine(f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_dbname}")
print("PostgreSQL engine created.")

PostgreSQL engine created.


### 2. Load Engineered Data into PostgreSQL

Load the final feature-engineered dataset into a PostgreSQL table called `tableau_insurance_claims`.  
Using `if_exists='replace'` ensures the table refreshes with the latest data on each run, keeping Tableau dashboards up-to-date.

In [3]:
# Drop the view if it exists using SQLAlchemy's `text()`
with engine.connect() as conn:
    conn.execute(text("DROP VIEW IF EXISTS v_fraud_summary;"))  # Wrapped in text()
    conn.commit()  # Commit the transaction

# Load the engineered dataset into a DataFrame
df = pd.read_csv(feature_file)

# Replace the PostgreSQL table
df.to_sql("tableau_insurance_claims", engine, if_exists="replace", index=False)
print(f"Loaded {df.shape[0]} records into PostgreSQL table: tableau_insurance_claims")

Loaded 1000 records into PostgreSQL table: tableau_insurance_claims


### 3. Manually Create PostgreSQL View for Tableau (Optional but Recommended)

To simplify Tableau data consumption, I have created a database view that selects and organizes the key columns from the fraud analytics table.

### Follow these steps to create the view using pgAdmin4:

1. **Open pgAdmin4** on your computer and connect to your PostgreSQL server.

2. In the left sidebar, navigate to your database:
   - Expand **Servers** → your server name → **Databases** → `fraud_analytics` (or your database name).

3. Right-click on **Schemas** → **public** → **Views**, and select **Create** → **View...**

4. In the **Create - View** dialog:
   - Enter the view name as: `v_fraud_summary`
   - Click the **Definition** tab or section.

5. Paste the following SQL query into the **View Definition** box:

    ```sql
    CREATE OR REPLACE VIEW v_fraud_summary AS
    SELECT
        fraud_reported,
        total_claim_amount,
        risk_score,
        fraud_weight,
        incident_year,
        incident_month,
        incident_dayofweek,
        incident_is_weekend,
        "incident_type_PARKED CAR",
        "incident_type_SINGLE VEHICLE COLLISION",
        "incident_type_VEHICLE THEFT",
        "collision_type_REAR COLLISION",
        "collision_type_SIDE COLLISION",
        "police_report_available_YES"
    FROM
    tableau_insurance_claims;
    ```

6. Click **Save** or **OK** to create the view.

7. To verify, expand the **Views** node under your database schema, and confirm `v_fraud_summary` appears.

8. Now, in Tableau, you can connect directly to this view `v_fraud_summary` instead of the full table for easier querying and cleaner dashboards.

---

> Notes:  
> Creating this view does **not** duplicate the data (as tested), it simply defines a saved SQL query that Tableau can use.  
> If data updates frequently, the view will always reflect the latest table data.  

---

### 4. Connect Tableau to PostgreSQL

Steps to connect Tableau Desktop to PostgreSQL data source:

1. Launch Tableau Desktop.
2. Select **PostgreSQL** as the connection type.
3. Enter connection details:
   - Server: `localhost`
   - Port: `5432`
   - Database: `fraud_analytics`
   - Username: `postgres`
   - Password: (your PostgreSQL password)
4. Choose your data source:
   - Table: `tableau_insurance_claims`  
   - Or View: `v_fraud_summary` (recommended for cleaner schema)
5. Click **Sheet** to begin creating dashboards.

---

### Dashboard Ideas for Fraud Analytics

- **Total Claim Amount** by Fraud Status  
- **Average Risk Score** by Incident Type  
- Fraud **Trends Over Time** (Monthly, Weekly, Daily)  
- Heatmap of **Fraud Frequency** by Hour of Day vs Collision Type  
- Bar chart showing **Fraud Percentage** by Police Report Availability